In [35]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torchsummaryX import summary

import numpy as np
import random
import math
import jieba

import sacrebleu
import torchtext
from typing import Tuple
import torchdata
import spacy
from collections import Counter  # 计数器，用于统计词频
from tqdm import tqdm  # 进度条库，用于显示进度


In [36]:

# 获取数据集文件路径
train_path = "E:/project_2023/11785/fanyi/data/train.txt"
val_path = "E:/project_2023/11785/fanyi/data/val.txt"
test_path = "E:/project_2023/11785/fanyi/data/test.txt"

# 定义批处理大小。
BATCH_SIZE = 128




SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [37]:


class TranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, source_tokenizer, target_tokenizer, source_vocab, target_vocab):
        """
        初始化数据集。
        source_sentences: 源语言句子列表。
        target_sentences: 目标语言句子列表。
        source_tokenizer: 源语言分词函数。
        target_tokenizer: 目标语言分词函数。
        source_vocab: 源语言词汇表。
        target_vocab: 目标语言词汇表。
        """
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences
        self.source_tokenizer = source_tokenizer
        self.target_tokenizer = target_tokenizer
        self.source_vocab = source_vocab
        self.target_vocab = target_vocab

    def __len__(self):
        # 确保源语言和目标语言数据列表的长度相等
        assert len(self.source_sentences) == len(self.target_sentences)
        # 返回数据集的大小
        return len(self.source_sentences)

    def __getitem__(self, index):
        # 获取单个句子
        source_sentence = self.source_sentences[index]
        target_sentence = self.target_sentences[index]
        
        # 对句子分词
        source_words = self.source_tokenizer(source_sentence)
        target_words = self.target_tokenizer(target_sentence)

        # 将分词结果转换为索引（如果词不在词汇表中，将使用 <unk> 的索引号）
        source_indices = [self.source_vocab.get(word, self.source_vocab['<unk>']) for word in source_words]
        target_indices = [self.target_vocab.get(word, self.target_vocab['<unk>']) for word in target_words]

        # 在序列的开始和结束分别添加特殊标记<sos>和<eos>
        source_indices = [self.source_vocab['<sos>']] + source_indices + [self.source_vocab['<eos>']]
        target_indices = [self.target_vocab['<sos>']] + target_indices + [self.target_vocab['<eos>']]

        # 将索引列表转换为PyTorch张量
        src_tensor = torch.tensor(source_indices, dtype=torch.long)
        tgt_tensor = torch.tensor(target_indices, dtype=torch.long)

        return src_tensor, tgt_tensor

    # 自定义批处理函数，用于数据加载器在加载批次数据时的数据处理
    def collate_fn(self, batch):
        # 将批次中的数据分解为源语言和目标语言张量列表
        src_tensors, tgt_tensors = zip(*batch)  # zip(*batch)：功能是将batch中一个个元组(src_tensor, tgt_tensor)先拆开，然后把所有src_tensor们组织到一起，把tgt_tensor们也组织到一起。
        # 使用pad_sequence对序列进行填充，使得批次中的所有序列长度相等，填充值为<pad>标记的索引
        src_tensors = torch.nn.utils.rnn.pad_sequence(src_tensors, padding_value=self.source_vocab['<pad>'], batch_first=True) # 当batch_first=True时，输出张量的形状将为 (batch_size, sequence_length, *)。如果不加，默认是False，输出维度为：(sequence_length, batch_size, *)
        tgt_tensors = torch.nn.utils.rnn.pad_sequence(tgt_tensors, padding_value=self.target_vocab['<pad>'], batch_first=True)

        # 返回处理好的源语言和目标语言批次数据
        return src_tensors, tgt_tensors



# 中文分词器
def tokenize_zh(text):
    """使用jieba分词来分割中文句子"""
    return list(jieba.cut(text))

# 英文分词器
def tokenize_en(text):
    return text.split() # 直接通过空格分隔句子


# 构建词汇表的函数
def build_tokenizer(lang_data, language='English'):
    # 初始化词汇表：pad代表空白填充、sos是开始标记、eos是结束标记、unk表示未知标记（主要用于没在训练集词汇表中出现过的词）
    word_to_index = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "<unk>": 3} 
    index = 4  # 从4开始编号，因为0-3已被特殊字符占用
    for sentence in lang_data:
        # 根据语言选择分词方法
        if language == 'Chinese':
            words = tokenize_zh(sentence)
        else:
            words = tokenize_en(sentence)
            
        for word in words:
            if word not in word_to_index:
                word_to_index[word] = index
                index += 1
    return word_to_index


# 从文件中读取数据
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().strip().split('\n')
    source_data = [line.split('\t')[0] for line in lines]
    target_data = [line.split('\t')[1] for line in lines]
    return source_data, target_data



# 从文件中加载英文和中文数据
train_data_en, train_data_zh = read_data(train_path)
val_data_en, val_data_zh = read_data(val_path)
test_data_en, test_data_zh = read_data(test_path)

# 创建英文和中文的词汇表
EN_VOCAB  = build_tokenizer(train_data_en, language='English')
ZH_VOCAB = build_tokenizer(train_data_zh, language='Chinese')
# 创建 index-to-word 字典
INDEX_TO_EN_VOCAB = {index: word for word, index in EN_VOCAB.items()}
# 创建 index-to-word 字典
INDEX_TO_ZH_VOCAB = {index: word for word, index in ZH_VOCAB.items()}


# 创建数据集实例
train_dataset = TranslationDataset(train_data_en, train_data_zh, tokenize_en, tokenize_zh, EN_VOCAB, ZH_VOCAB)
val_dataset = TranslationDataset(val_data_en, val_data_zh, tokenize_en, tokenize_zh, EN_VOCAB, ZH_VOCAB)
test_dataset = TranslationDataset(test_data_en, test_data_zh, tokenize_en, tokenize_zh, EN_VOCAB, ZH_VOCAB)

# 创建DataLoader对象
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=val_dataset.collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=test_dataset.collate_fn)

In [38]:
# 打印原文本和目标文本
sample_source_text = train_data_en[280]
sample_target_text = train_data_zh[280]
print("原文本:", sample_source_text)
print("目标文本:", sample_target_text)

# 打印分词后的原文本和目标文本
print("分词后的原文本:", tokenize_en(sample_source_text))
print("分词后的目标文本:", tokenize_zh(sample_target_text))

# 显示英文词汇表的一部分
print("英文词汇表个数:", len(EN_VOCAB))
print("英文词汇表样本:", list(EN_VOCAB.items())[:10])
# 显示中文词汇表的一部分
print("中文词汇表个数:", len(ZH_VOCAB))
print("中文词汇表样本:", list(ZH_VOCAB.items())[:10])


原文本: i will write to you soon . 
目标文本: 我会尽快写信给你。
分词后的原文本: ['i', 'will', 'write', 'to', 'you', 'soon', '.']
分词后的目标文本: ['我会', '尽快', '写信给', '你', '。']
英文词汇表个数: 5581
英文词汇表样本: [('<pad>', 0), ('<sos>', 1), ('<eos>', 2), ('<unk>', 3), ('it', 4), ("'s", 5), ('none', 6), ('of', 7), ('your', 8), ('concern', 9)]
中文词汇表个数: 9120
中文词汇表样本: [('<pad>', 0), ('<sos>', 1), ('<eos>', 2), ('<unk>', 3), ('这不关', 4), ('你', 5), ('的', 6), ('事', 7), ('。', 8), ('她', 9)]


# 模型构建

In [39]:

# PositionalEncoding类继承自nn.Module，是一个PyTorch模块
class PositionalEncoding(nn.Module):

    # 类的初始化方法
    def __init__(self, d_model, max_seq_len, dropout=0.1):
        """
        初始化PositionalEncoding模块。

        参数:
        d_model (int): 输入向量的维度。
        max_seq_len (int): 输入序列的最大长度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        # 调用父类的初始化方法
        super(PositionalEncoding, self).__init__()
        # 定义一个dropout层，用于在位置编码加到输入向量之后进行dropout操作
        self.dropout = nn.Dropout(dropout)

        # 初始化一个位置编码矩阵，全为零，形状为(max_seq_len, d_model)
        pe = torch.zeros(max_seq_len, d_model)

        # 生成一个位置索引向量，形状为(max_seq_len, 1)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)

        # 生成除数项，用于计算正弦和余弦函数的参数，形状为(d_model / 2,)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))

        # 计算位置编码的正弦部分，填充到偶数位置
        pe[:, 0::2] = torch.sin(position * div_term)
        # 计算位置编码的余弦部分，填充到奇数位置
        pe[:, 1::2] = torch.cos(position * div_term)
        # 增加一个维度，使pe的形状为(1, max_seq_len, d_model)，方便后续的广播操作
        pe = pe.unsqueeze(0)

        # 将位置编码矩阵注册为一个buffer，这样它就可以被保存在模型的state_dict中了，
        # 虽然它不是一个可学习的参数。
        self.register_buffer('pe', pe)

    # 定义前向传播方法
    def forward(self, x):
        """
        PositionalEncoding的前向传播方法。
        
        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 将位置编码加到输入张量上，使用广播机制匹配序列长度
        x = x + self.pe[:, :x.size(1), :]
        # 对加上位置编码后的输入进行dropout操作
        x = self.dropout(x)
        
        # 返回处理后的张量
        return x
    



# AddNorm类继承自nn.Module，是PyTorch中的标准写法
class AddNorm(nn.Module):

    def __init__(self, d_model, eps=1e-6):
        """
        初始化AddNorm模块。

        参数:
        d_model (int): 输入张量的维度。
        eps (float, optional): 为了数值稳定性而加入的一个小常数，默认值为1e-6。
        """
        super(AddNorm, self).__init__()  # 调用父类的初始化函数
        # 初始化一个层归一化（LayerNorm）模块，它会对输入张量的最后一维进行归一化
        self.norm = nn.LayerNorm(d_model, eps=eps)

    def forward(self, x, residual):
        """
        AddNorm的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。
        residual (Tensor): 残差张量，与输入张量形状相同。

        返回:
        Tensor: 输出张量，形状也为(batch_size, seq_length, d_model)。
        """
        # 首先进行残差连接，即将输入张量和残差张量相加
        out = x + residual
        # 然后对相加后的结果进行层归一化处理
        out = self.norm(out)

        # 返回归一化后的输出张量
        return out

    
    
    
    
# 定义多头自注意力模块，继承自nn.Module
class MultiHeadSelfAttention(nn.Module):

    def __init__(self, d_model, num_heads):
        """
        初始化多头自注意力模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力头的数量。
        """
        super(MultiHeadSelfAttention, self).__init__()
        self.d_model = d_model  # 保存输入维度
        self.num_heads = num_heads  # 保存头的数量
        self.head_dim = d_model // num_heads  # 计算每个头的维度

        # 确保d_model能被num_heads整除，保证等分
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"

        # 定义全连接层，用于生成查询、键、值
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)

        # 定义输出全连接层
        self.wo = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        """
        多头自注意力的前向传播。

        参数:
        q (Tensor): 查询张量，形状为(batch_size, seq_length, d_model)。
        k (Tensor): 键张量，形状为(batch_size, seq_length, d_model)。
        v (Tensor): 值张量，形状为(batch_size, seq_length, d_model)。
        mask (Tensor, optional): 掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        batch_size = q.size(0)

        # 将查询、键、值通过全连接层后，改变形状以适应多头处理
        q = self.wq(q).view(batch_size, -1, self.num_heads, self.head_dim)
        k = self.wk(k).view(batch_size, -1, self.num_heads, self.head_dim)
        v = self.wv(v).view(batch_size, -1, self.num_heads, self.head_dim)

        # 转置操作，以满足矩阵乘法的需求
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # 计算注意力权重矩阵
        attn = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)

        # 如果提供了掩码，则应用掩码
        if mask is not None:
            attn = attn.masked_fill(mask == 0, float('-inf'))

        # 应用softmax获取标准化的注意力权重
        attn = F.softmax(attn, dim=-1)

        # 根据注意力权重计算输出值
        out = torch.matmul(attn, v).transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        # 将输出通过最后一个全连接层
        out = self.wo(out)

        return out
    
# PositionwiseFeedForward类继承自nn.Module，是一个PyTorch模块
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        初始化逐位置前馈网络模块。

        参数:
        d_model (int): 输入张量的维度。
        d_ff (int): 前馈网络隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(PositionwiseFeedForward, self).__init__()
        # 定义第一个全连接层，将输入维度从d_model映射到d_ff
        self.linear1 = nn.Linear(d_model, d_ff)
        # 定义一个dropout层，用于防止过拟合
        self.dropout = nn.Dropout(dropout)
        # 定义第二个全连接层，将维度从d_ff映射回d_model
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        """
        逐位置前馈网络的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过第一个全连接层
        out = self.linear1(x)
        # 应用ReLU激活函数
        out = F.relu(out)
        # 应用dropout
        out = self.dropout(out)
        # 通过第二个全连接层
        out = self.linear2(out)

        # 返回最终的输出张量
        return out


# EncoderBlock类继承自nn.Module，是一个PyTorch模块
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        初始化编码器块模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(EncoderBlock, self).__init__()
        # 初始化多头自注意力层
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第一个添加残差连接和层归一化的模块
        self.norm1 = AddNorm(d_model)
        # 初始化逐位置前馈网络
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        # 初始化第二个添加残差连接和层归一化的模块
        self.norm2 = AddNorm(d_model)

    def forward(self, x, mask=None):
        """
        编码器块的前向传播函数。

        参数:
        x (Tensor): 输入张量，形状为(batch_size, seq_length, d_model)。seq_length取当前批次最长句子的长度值。
        mask (Tensor, optional): 掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过多头自注意力层处理输入
        x1 = self.self_attn(q=x, k=x, v=x, mask=mask)
        # 应用残差连接和层归一化
        x = self.norm1(x, x1)
        # 通过逐位置前馈网络处理
        x1 = self.ffn(x)
        # 再次应用残差连接和层归一化
        x = self.norm2(x, x1)

        # 返回最终的输出
        return x


# DecoderBlock类继承自nn.Module，是一个PyTorch模块
class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        初始化解码器块模块。

        参数:
        d_model (int): 输入张量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(DecoderBlock, self).__init__()
        # 初始化自注意力层
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第一个残差连接和层归一化
        self.norm1 = AddNorm(d_model)
        # 初始化编码器-解码器注意力层，用于关注编码器的输出
        self.enc_dec_attn = MultiHeadSelfAttention(d_model, num_heads)
        # 初始化第二个残差连接和层归一化
        self.norm2 = AddNorm(d_model)
        # 初始化逐位置前馈网络
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        # 初始化第三个残差连接和层归一化
        self.norm3 = AddNorm(d_model)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        """
        解码器块的前向传播函数。

        参数:
        x (Tensor): 目标输入张量，形状为(batch_size, seq_length, d_model)。
        enc_output (Tensor): 编码器输出张量，形状为(batch_size, seq_length, d_model)。
        src_mask (Tensor, optional): 来源掩码张量，用于忽略某些元素，针对编码器输出，默认为None。
        tgt_mask (Tensor, optional): 目标掩码张量，用于忽略某些元素，针对自注意力层，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, seq_length, d_model)。
        """
        # 通过自注意力层处理输入
        x1 = self.self_attn(q=x, k=x, v=x, mask=tgt_mask)
        # 应用残差连接和层归一化
        x = self.norm1(x, x1)
        # 通过编码器-解码器注意力层处理，关注编码器的输出
        x1 = self.enc_dec_attn(q=x, k=enc_output, v=enc_output, mask=src_mask)
        # 再次应用残差连接和层归一化
        x = self.norm2(x, x1)
        # 通过逐位置前馈网络处理
        x1 = self.ffn(x)
        # 最后再次应用残差连接和层归一化
        x = self.norm3(x, x1)

        # 返回最终的输出
        return x


# Transformer类继承自nn.Module，是一个PyTorch模块
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, D_MODEL, num_heads, d_ff, max_seq_len, num_layers, dropout=0.1):
        """
        初始化Transformer模块。

        参数:
        src_vocab_size (int): 源词汇表的大小。
        tgt_vocab_size (int): 目标词汇表的大小。
        D_MODEL (int): 嵌入向量的维度。
        num_heads (int): 注意力机制中头的数量。
        d_ff (int): 前馈网络中隐藏层的维度。
        max_seq_len (int): 输入序列的最大长度。
        num_layers (int): 编码器和解码器中层的数量。
        dropout (float, optional): Dropout概率，默认为0.1。
        """
        super(Transformer, self).__init__()
        # 初始化源词嵌入层
        self.src_embedding = nn.Embedding(src_vocab_size, D_MODEL)
        # 初始化目标词嵌入层
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, D_MODEL)
        # 初始化位置编码
        self.pos_encoding = PositionalEncoding(D_MODEL, max_seq_len, dropout)

        # 初始化编码器层，使用ModuleList容纳多个EncoderBlock实例
        self.encoder_layers = nn.ModuleList([EncoderBlock(D_MODEL, num_heads, d_ff, dropout) for _ in range(num_layers)])
        # 初始化解码器层，同样使用ModuleList
        self.decoder_layers = nn.ModuleList([DecoderBlock(D_MODEL, num_heads, d_ff, dropout) for _ in range(num_layers)])

        # 初始化一个全连接层，用于将解码器的输出映射到目标词汇空间
        self.fc = nn.Linear(D_MODEL, tgt_vocab_size)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        Transformer的前向传播函数。

        参数:
        src (Tensor): 源输入张量，形状为(batch_size, src_seq_length)。src_seq_length取当前批次中长度最大的那个句子长度。
        tgt (Tensor): 目标输入张量，形状为(batch_size, tgt_seq_length)。
        src_mask (Tensor, optional): 源掩码张量，用于忽略某些元素，默认为None。
        tgt_mask (Tensor, optional): 目标掩码张量，用于忽略某些元素，默认为None。

        返回:
        Tensor: 输出张量，形状为(batch_size, tgt_seq_length, tgt_vocab_size)。
        """
        # 对源序列进行词嵌入和位置编码
        src = self.src_embedding(src) #[batch_size, src_seq_length]->[bs, s_s_l, d_model]
        src = self.pos_encoding(src)

        # 对目标序列进行词嵌入和位置编码
        tgt = self.tgt_embedding(tgt)
        tgt = self.pos_encoding(tgt)
        
        
        
        #测试对解码器的输入tgt进行mask，防止残差泄露信息
        # 调整掩码形状
        #tgt = tgt.masked_fill(mask == 0, float('-inf'))


        # 依次通过编码器层
        for layer in self.encoder_layers:
            src = layer(src, src_mask)

        # 依次通过解码器层，需要提供编码器的输出和掩码信息
        for layer in self.decoder_layers:
            tgt = layer(tgt, src, src_mask, tgt_mask)

        # 通过最后的全连接层，映射到目标词汇空间
        out = self.fc(tgt)

        return out    



In [40]:
# 定义训练时的超参数
NUM_EPOCHS      = 10            # 训练的轮数
D_MODEL         = 256           # 模型中嵌入向量的维度
ATTN_HEADS      = 8             # 多头注意力中头的数量
NUM_LAYERS      = 3             # 编码器和解码器层重复的次数
FEEDFORWARD_DIM = 512           # 前馈网络中隐藏层的维度
DROPOUT         = 0.1           # Dropout概率，用于防止过拟合
MAX_SEQ_LEN     = 150           # 输入序列的最大长度
SRC_VOCAB_SIZE  = len(EN_VOCAB)  # 源语言词汇表的大小
TGT_VOCAB_SIZE  = len(ZH_VOCAB)  # 目标语言词汇表的大小
LR              = 0             # 学习率，这里设置为0可能是为了后续调整


# 设备配置，优先使用GPU，如果没有GPU则使用CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 负责根据训练步骤动态调整学习率
class NoamScheduler:

    def __init__(self, optimizer, d_model, warmup_steps=4000):
        """
        初始化NoamScheduler。

        参数:
        optimizer: 用于训练的优化器。
        d_model (int): 模型中嵌入向量的维度。
        warmup_steps (int): 预热步数，在此步数之前学习率会线性增加。
        """
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.current_step = 0  # 初始化当前步数

    def step(self):
        """
        更新学习率并增加步数。
        """
        self.current_step += 1  # 每调用一次，步数增加1
        lr = self.learning_rate()  # 计算当前步数的学习率
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr  # 更新优化器中的学习率

    def learning_rate(self):
        """
        根据当前步数计算学习率。
        """
        step = self.current_step
        # 学习率随着步数先增后减，增加部分线性增加至warmup_steps，之后随步数的增加而减小
        return (self.d_model ** -0.5) * min(step ** -0.5, step * self.warmup_steps ** -1.5)


# 初始化Transformer模型，并将其移动到适当的设备上
model = Transformer(SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, D_MODEL, ATTN_HEADS, FEEDFORWARD_DIM, MAX_SEQ_LEN, NUM_LAYERS, DROPOUT).to(DEVICE)

# 初始化优化器，这里使用AdamW优化器，它是Adam优化器的一个变种，添加了权重衰减
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9, weight_decay=5e-2)

# 根据训练数据加载器的长度计算预热步数
warmup_steps = 2 * len(train_dataloader)

# 使用NoamScheduler作为学习率调度器
scheduler = NoamScheduler(optimizer, d_model=D_MODEL, warmup_steps=warmup_steps)

# 初始化损失函数，使用交叉熵损失，并忽略'<pad>'标记的损失，同时应用标签平滑
criterion = torch.nn.CrossEntropyLoss(ignore_index=ZH_VOCAB['<pad>'], label_smoothing=0.1)

# 初始化梯度缩放器，用于混合精度训练，有助于提高训练速度和减少内存消耗
scaler = torch.cuda.amp.GradScaler()


import sacrebleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

def generate_tgt_mask(tgt, pad_idx):
    """
    生成目标序列的掩码，遮蔽未来位置以及填充位置。
    
    参数:
    tgt (Tensor): 目标序列。
    pad_idx (int): `<pad>`标记的索引。
    
    返回:
    Tensor: 组合掩码。
    """
    seq_len = tgt.size(1)
    # 生成一个下三角矩阵，用于屏蔽未来的信息
    no_future_mask = torch.tril(torch.ones((seq_len, seq_len), device=DEVICE)).bool()
    # 生成一个填充位置的掩码
    pad_mask = (tgt != pad_idx).unsqueeze(1).unsqueeze(2)
    # 将两个掩码相与，得到最终的掩码
    combined_mask = pad_mask & no_future_mask
    return combined_mask


def generate_src_mask(src, pad_idx):
    """
    生成源序列的掩码，遮蔽填充位置。
    
    参数:
    src (Tensor): 源序列。
    pad_idx (int): `<pad>`标记的索引。
    
    返回:
    Tensor: 源序列的掩码。
    """
    mask = (src != pad_idx).unsqueeze(1).unsqueeze(2)
    return mask


def calculate_bleu(tgt_output, output):
    """
    计算BLEU分数。
    
    参数:
    tgt_output (Tensor): 真实的目标序列。维度：[batchsize, 句子最大长度]
    output (Tensor): 模型生成的输出序列。维度：[batchsize, 句子最大长度]
    
    返回:
    float: BLEU分数。
    """
    tgt_output = tgt_output.cpu().numpy()
    output = output.cpu().numpy()

    bleu = 0
    
    # 需要排除的标记
    excluded_tokens = (ZH_VOCAB['<pad>'], ZH_VOCAB['<eos>'], ZH_VOCAB['<sos>'])
    for tgt, pred in zip(tgt_output, output):
        # 将索引转换为单词，排除特定标记
        ref = ' '.join([INDEX_TO_ZH_VOCAB.get(t, '<unk>') for t in tgt if t not in excluded_tokens])
        hyp = ' '.join([INDEX_TO_ZH_VOCAB.get(t, '<unk>') for t in pred if t not in excluded_tokens])
        
        bleu += sacrebleu.corpus_bleu([hyp], [[ref]]).score

    # 平均BLEU分数（批放进去，数据结果有点难懂，所以用单个句子放进去求平均）
    return bleu/len(tgt_output)


src, tgt = next(iter(train_dataloader))
src, tgt = src.to(DEVICE), tgt.to(DEVICE)

src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])

tgt_input = tgt[:, :-1]
tgt_output = tgt[:, 1:]
tgt_mask = generate_tgt_mask(tgt_input, ZH_VOCAB['<pad>'])

summary(model, src, tgt_input, src_mask, tgt_mask)


                                           Kernel Shape     Output Shape  \
Layer                                                                      
0_src_embedding                             [256, 5581]   [128, 19, 256]   
1_pos_encoding.Dropout_dropout                        -   [128, 19, 256]   
2_tgt_embedding                             [256, 9120]   [128, 15, 256]   
3_pos_encoding.Dropout_dropout                        -   [128, 15, 256]   
4_encoder_layers.0.self_attn.Linear_wq       [256, 256]   [128, 19, 256]   
5_encoder_layers.0.self_attn.Linear_wk       [256, 256]   [128, 19, 256]   
6_encoder_layers.0.self_attn.Linear_wv       [256, 256]   [128, 19, 256]   
7_encoder_layers.0.self_attn.Linear_wo       [256, 256]   [128, 19, 256]   
8_encoder_layers.0.norm1.LayerNorm_norm           [256]   [128, 19, 256]   
9_encoder_layers.0.ffn.Linear_linear1        [256, 512]   [128, 19, 512]   
10_encoder_layers.0.ffn.Dropout_dropout               -   [128, 19, 512]   
11_encoder_l

e:\soft_big\anaconda3\envs\torch12\lib\site-packages\torchsummaryX\torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_src_embedding,"[256, 5581]","[128, 19, 256]",1428736.0,1428736.0
1_pos_encoding.Dropout_dropout,-,"[128, 19, 256]",NaN,NaN
2_tgt_embedding,"[256, 9120]","[128, 15, 256]",2334720.0,2334720.0
3_pos_encoding.Dropout_dropout,-,"[128, 15, 256]",NaN,NaN
4_encoder_layers.0.self_attn.Linear_wq,"[256, 256]","[128, 19, 256]",65792.0,65536.0
...,...,...,...,...
69_decoder_layers.2.ffn.Linear_linear1,"[256, 512]","[128, 15, 512]",131584.0,131072.0
70_decoder_layers.2.ffn.Dropout_dropout,-,"[128, 15, 512]",NaN,NaN
71_decoder_layers.2.ffn.Linear_linear2,"[512, 256]","[128, 15, 256]",131328.0,131072.0


In [41]:

def train_epoch(model, dataloader, optimizer, criterion, device):
    """
    对模型进行一个epoch的训练。

    参数:
    model: 训练的模型。
    dataloader: 数据加载器，提供训练数据。
    optimizer: 优化器，用于更新模型参数。
    criterion: 损失函数。
    device: 训练使用的设备（CPU或GPU）。

    返回:
    float: 该epoch的平均损失。
    """
    model.train()  # 将模型设置为训练模式
    total_loss = 0  # 记录总损失

    # 使用tqdm库显示训练进度条
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True,
                     leave=False, position=0, desc='Train')

    for i, (src, tgt) in enumerate(dataloader):
        # 将数据移动到指定的设备上
        src, tgt = src.to(device), tgt.to(device)
        # 生成源序列和目标序列的掩码
        src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])
        tgt_input = tgt[:, :-1] #去除最末尾的sos标志
        tgt_output = tgt[:, 1:] #去除最开始的eos标志
        tgt_mask = generate_tgt_mask(tgt_input, ZH_VOCAB['<pad>'])

        optimizer.zero_grad()  # 清空梯度

        # 使用混合精度训练
        with torch.cuda.amp.autocast():
            output = model(src, tgt_input, src_mask, tgt_mask)
            loss = criterion(output.reshape(-1, output.size(2)), tgt_output.reshape(-1))

        # 反向传播和优化
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        scheduler.step()  # 更新学习率

        total_loss += loss.item()
        # 更新进度条
        batch_bar.set_postfix(
            loss="{:.04f}".format(total_loss / (i + 1)),
            lr="{:.09f}".format(float(optimizer.param_groups[0]['lr'])))
        batch_bar.update()

    return total_loss / len(dataloader)


def validate_epoch(model, dataloader, criterion, DEVICE):
    """
    对模型进行一个epoch的验证。

    参数:
    model: 需要验证的模型。
    dataloader: 数据加载器，提供验证数据。
    criterion: 损失函数。
    DEVICE: 验证使用的设备（CPU或GPU）。

    返回:
    tuple: 包含平均损失和平均BLEU分数的元组。
    """
    model.eval()  # 将模型设置为评估模式
    epoch_loss = 0
    epoch_bleu_score = 0

    # 使用tqdm库显示验证进度条
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True,
                     leave=False, position=0, desc='Validate')

    with torch.no_grad():  # 在验证过程中不计算梯度
        for i, (src, tgt) in enumerate(dataloader):
            src, tgt = src.to(DEVICE), tgt.to(DEVICE)
            src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]
            tgt_mask = generate_tgt_mask(tgt_input, ZH_VOCAB['<pad>'])

            # 使用混合精度评估
            with torch.cuda.amp.autocast():
                output = model(src, tgt_input, src_mask, tgt_mask)
                loss = criterion(output.reshape(-1, output.shape[-1]), tgt_output.reshape(-1))

            epoch_loss += loss.item()
            # 计算并累加BLEU分数
            epoch_bleu_score += calculate_bleu(tgt_output, output.argmax(-1))

            # 更新进度条
            batch_bar.set_postfix(
                loss="{:.04f}".format(epoch_loss / (i + 1)),
                bleu="{:.04f}".format(epoch_bleu_score / (i + 1)))
            batch_bar.update()

    # 计算平均损失和BLEU分数
    epoch_loss /= len(dataloader)
    epoch_bleu_score /= len(dataloader)

    return epoch_loss, epoch_bleu_score





best_val_loss = float('inf')  # 记录最佳验证损失值
train_losses = []  # 存储每个epoch的训练损失
val_losses = []  # 存储每个epoch的验证损失
bleu_scores = []  # 存储每个epoch的BLEU分数

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"Epoch {epoch}/{NUM_EPOCHS}")

    # 训练
    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, DEVICE)
    train_losses.append(train_loss)

    # 验证
    val_loss, bleu_score = validate_epoch(model, val_dataloader, criterion, DEVICE)
    val_losses.append(val_loss)
    bleu_scores.append(bleu_score)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | BLEU Score: {bleu_score:.4f}")

    # 如果当前验证损失低于之前的最佳值，则保存模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 1/10


Train Loss: 5.7974 | Val Loss: 4.6357 | BLEU Score: 3.9801
Epoch 2/10


Train Loss: 4.2748 | Val Loss: 4.0811 | BLEU Score: 5.3245
Epoch 3/10


Train Loss: 3.6974 | Val Loss: 3.7223 | BLEU Score: 6.7606
Epoch 4/10


Train Loss: 3.1673 | Val Loss: 3.5713 | BLEU Score: 7.3285
Epoch 5/10


...

Train Loss: 2.4464 | Val Loss: 3.4390 | BLEU Score: 8.2231
Epoch 7/10


Train Loss: 2.2069 | Val Loss: 3.3831 | BLEU Score: 8.8944
Epoch 8/10


Train Loss: 2.0361 | Val Loss: 3.3935 | BLEU Score: 9.3932
Epoch 9/10


Train Loss: 1.9146 | Val Loss: 3.4245 | BLEU Score: 9.4608
Epoch 10/10


Train Loss: 1.8221 | Val Loss: 3.4423 | BLEU Score: 9.5774


In [45]:
# 创建 index-to-word 字典
INDEX_TO_EN_VOCAB = {index: word for word, index in EN_VOCAB.items()}
# 创建 index-to-word 字典
INDEX_TO_ZH_VOCAB = {index: word for word, index in ZH_VOCAB.items()}
def inference(model, src, de_tokenizer):
    """
    使用训练好的模型进行推理，生成目标语言序列。

    参数:
    model: 训练好的Transformer模型。
    src (Tensor): 源语言序列的张量。
    de_tokenizer: 目标语言的词汇表，用于将索引转换为单词。

    返回:
    list: 生成的目标语言句子列表。
    """
    model.eval()  # 将模型设置为评估模式

    src_mask = generate_src_mask(src, EN_VOCAB['<pad>'])  # 生成源序列的掩码

    # 使用<sos>标记初始化目标输入张量
    tgt_input = torch.full((src.size(0), 1), ZH_VOCAB['<sos>'], dtype=torch.long, device=DEVICE)

    # 为批处理中的每个序列创建一个结束标志
    eos_flags = torch.zeros(src.size(0), dtype=torch.bool, device=DEVICE)

    # 对每个目标令牌进行推理
    with torch.no_grad():
        for _ in range(70):  # 最多生成70个令牌
            tgt_mask = generate_tgt_mask(tgt_input, ZH_VOCAB['<pad>'])  # 生成目标序列的掩码
            output = model(src, tgt_input, src_mask, tgt_mask)
            next_tokens = output.argmax(2)[:, -1].unsqueeze(1)  # 选择概率最高的下一个令牌
            tgt_input = torch.cat((tgt_input, next_tokens), dim=1)  # 将下一个令牌添加到目标输入中

            # 更新已生成<eos>标记的序列的结束标志
            eos_flags |= (next_tokens.squeeze() == ZH_VOCAB['<eos>'])
            # 如果所有序列都已生成<eos>或达到最大长度，则停止生成
            if torch.all(eos_flags):
                break

    # 将目标输入张量转换为翻译后的句子
    translated_sentences = []
    for i in range(tgt_input.size(0)):
        translated_tokens = []
        for token in tgt_input[i][1:]:  # 跳过第一个<sos>标记
            if token.item() == ZH_VOCAB['<eos>']: # 遇到<eos>标记对应的索引值时停止
                break  
            else:
                translated_tokens.append(INDEX_TO_ZH_VOCAB.get(token.item(), '<unk>'))  # 将索引转换为单词, 找不到的单词置为<unk>。

        translated_sentence = ' '.join(translated_tokens)  # 将单词列表连接成句子
        translated_sentences.append(translated_sentence)

    return translated_sentences


def evaluate_test_set_bleu(model, test_dataloader, de_tokenizer):
    """
    在测试集上评估模型的BLEU分数。

    参数:
    model: 训练好的Transformer模型。
    test_dataloader: 测试数据的数据加载器。
    de_tokenizer: 目标语言的词汇表。

    返回:
    float: 测试集的BLEU分数。
    """
    n = 0
    bleu = 0
    for batch in tqdm(test_dataloader, desc="Evaluating"):
        src, tgt_output = batch
        src, tgt = src.to(DEVICE), tgt_output.to(DEVICE)
        tgt_sentences = [' '.join([INDEX_TO_ZH_VOCAB.get(token.item(), '<unk>') for token in sequence if token.item() not in [ZH_VOCAB['<pad>'], ZH_VOCAB['<sos>'], ZH_VOCAB['<eos>']]]) for sequence in tgt_output]

        translations = inference(model, src, de_tokenizer)
        
        bleu_score2 = sacrebleu.corpus_bleu([translations[0]],  [[tgt_sentences[0]]])
        
        for i in range(len(translations)):
            bleu += sacrebleu.corpus_bleu([translations[i]], [[tgt_sentences[i]]]).score
        
        print("\n test集第" + str(n*128) +'句话：')
        print("输入英语     :", ' '.join([INDEX_TO_EN_VOCAB.get(i.item(), '<unk>') for i in src[0] if INDEX_TO_EN_VOCAB.get(i.item()) not in ['<sos>', '<eos>', '<pad>'] ])) # i是一个tensor
        print("GT          :", tgt_sentences[0])
        print("模型预测     :", translations[0])
        print("该句话BLEU得分：{:.2f}".format(bleu_score2.score))
        n += 1

    return bleu/len(test_dataset)

# Usage example
test_bleu = evaluate_test_set_bleu(model, test_dataloader, ZH_VOCAB)
print("test集每个句子的平均 BLEU 分数:", test_bleu)

Evaluating:   5%|▍         | 1/21 [00:00<00:06,  3.10it/s]


 test集第0句话：
输入英语     : do n't underestimate my power .
GT          : 不要 小看 我 的 力量 。
模型预测     : 不要 低估 我 的 力量 。
该句话BLEU得分：53.73


Evaluating:  14%|█▍        | 3/21 [00:00<00:03,  4.70it/s]


 test集第128句话：
输入英语     : do you still want to talk to me ?
GT          : 你 还 想 跟 我 谈 吗 ？
模型预测     : 你 还 想 跟 我 说 吗 ？
该句话BLEU得分：59.46

 test集第256句话：
输入英语     : give us a ride downtown .
GT          : 载 我们 到 市区 。
模型预测     : 把 一个 送 我们 去 市里 。
该句话BLEU得分：7.81

 test集第384句话：
输入英语     : my shoulder really aches .
GT          : 我 的 肩膀 很 <unk> 。
模型预测     : 我 的 再见 。
该句话BLEU得分：13.01


Evaluating:  29%|██▊       | 6/21 [00:01<00:02,  5.83it/s]

...


 test集第1920句话：
输入英语     : i 'm counting on your help .
GT          : 我 指望 你 的 帮助 。
模型预测     : 我 在 帮忙 。
该句话BLEU得分：11.52

 test集第2048句话：
输入英语     : i used to go out with friends every weekend .
GT          : 我 曾经 每 周末 都 和 朋友 外出 。
模型预测     : 我 以前 上 牀 朋友 。
该句话BLEU得分：6.48


Evaluating:  95%|█████████▌| 20/21 [00:03<00:00,  6.83it/s]


 test集第2176句话：
输入英语     : we accept <unk> .
GT          : 我们 接受 支票 。
模型预测     : 我们 接受 了 。
该句话BLEU得分：35.36

 test集第2304句话：
输入英语     : my <unk> turned out to be right .
GT          : 我 的 预感 被 证明 是 正确 的 。
模型预测     : 我 的 猜想 证明 是 正确 的 。
该句话BLEU得分：52.47

 test集第2432句话：
输入英语     : may i see you in private ?
GT          : 我们 能 私下 见见 吗 ？
模型预测     : 我 可以 在 你 私下 吗 ？
该句话BLEU得分：14.54


Evaluating: 100%|██████████| 21/21 [00:03<00:00,  5.83it/s]


 test集第2560句话：
输入英语     : i 'm a stranger here myself . i 'm afraid i can not help you .
GT          : 我 对 这里 不 熟 。 恐怕 我 不能 帮 你 。
模型预测     : 我 害怕 这儿 的 别人 ， 我 无法 自己 一个 人 。
该句话BLEU得分：4.46
test集每个句子的平均 BLEU 分数: 24.267820902819505
